<a href="https://colab.research.google.com/github/deepak1195/DeepLearning/blob/main/008_SingelNeuronNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path="/content/drive/MyDrive/Study/Data/"

Mounted at /content/drive


### Loading Data

In [ ]:
df=pd.read_csv(f'{path}insuranceData.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


### Normalizing Data

In [ ]:
df['age']=df['age']/100
df.head()

,age,affordibility,bought_insurance
0,0.22,1,0
1,0.25,0,0
2,0.47,1,1
3,0.52,0,0
4,0.46,1,1


### Spliting Data Between Train And Test

In [ ]:
from sklearn.model_selection import train_test_split
xTrain,xTest,yTrain,yTest=train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=25)

### Single Neuron

In [ ]:
class SNN():
  def __init__(self):
    self.weigPlusBias=None
  def logLoss(self,yT,yP):
    eplison=1e-15
    nYp=[max(i,eplison) for i in yP]
    nYp=np.array([min(i,1-eplison) for i in nYp])
    return -np.mean(yT*np.log(nYp)+(1-yT)*np.log(1-nYp))

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))

  def gradDesc(self, xT,yT,epochs,lThr):
    ws=np.ones(xT.shape[1])
    xs=np.array(xT.columns)
    b=np.zeros(xT.shape[0])
    r=0.5
    n=len(yT)
    for e in range(epochs):
      eq=[b]
      for i,j in zip(range(0,len(xs)),xs):
        eq.append(np.array(ws[i]*xT[[j]].values).reshape([1, n])[0])

      weigSum=np.sum(np.array(eq), axis = 0)
      yP=self.sigmoid(weigSum)
      loss=self.logLoss(yT,yP)

      for w,x in zip(range(len(ws)),xs):
        ws[w]=ws[w]-r*((1/n)*np.dot(np.transpose(xT[x]),(yP-yT)))
      b=b-r*(np.mean(yP-yT))

      if e%50==0:
        print(f"Epoch : {e}/{epochs}| Weights : {ws}| Bias : {b[0]}| Loss : {loss}")

      if loss<=lThr:
        print(f"Epoch : {e}/{epochs}| Weights : {ws}| Bias : {b[0]}| Loss : {loss}")
        break

    return np.append(ws,b[0])

  def fit(self,x,y,ep,lTh):
    self.weigPlusBias=self.gradDesc(x,y,ep,lTh)

  def predict(self,xTest):
    n=len(xTest)
    pws=[]
    for i,j in zip(range(0,len(xTest)),xTest):
      pws.append(np.array(self.weigPlusBias[i]*xTest[j].values))

    pws.append(np.ones(xTest.shape[0])*self.weigPlusBias[-1])
    print(self.sigmoid(np.sum(np.array(pws), axis = 0)))

In [ ]:
n=SNN()

In [ ]:
n.fit(xTrain,yTrain,1000,0.4631)

Epoch : 0/1000| Weights : [0.97490763 0.94834813]| Bias : -0.11341867736368583| Loss : 0.7113403233723417
Epoch : 50/1000| Weights : [1.50331955 1.10838479]| Bias : -1.2319047301235464| Loss : 0.5675865113475955
Epoch : 100/1000| Weights : [2.20071313 1.2941584 ]| Bias : -1.6607009122062801| Loss : 0.5390680417774752
Epoch : 150/1000| Weights : [2.84957278 1.36968955]| Bias : -1.986105845859897| Loss : 0.5176462164249294
Epoch : 200/1000| Weights : [3.44301697 1.40422186]| Bias : -2.2571369883752723| Loss : 0.5005011269691375
Epoch : 250/1000| Weights : [3.98245049 1.42391273]| Bias : -2.494377365971801| Loss : 0.48654089537617085
Epoch : 300/1000| Weights : [4.47217952 1.43878799]| Bias : -2.707387811922373| Loss : 0.4750814640632793
Epoch : 350/1000| Weights : [4.91724587 1.45256608]| Bias : -2.9011763335567657| Loss : 0.46561475306999006
Epoch : 366/1000| Weights : [5.05104762 1.45697945]| Bias : -2.9596534546250033| Loss : 0.4629394409588892


In [ ]:
print(n.weigPlusBias)

[ 5.05104762  1.45697945 -2.95965345]


In [ ]:
n.predict(xTest)

[0.70501995 0.35583554 0.16159866 0.4779192  0.72558553 0.82898702]


In [ ]:
yTest

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64